# Nilearn tutorial 1

## 1. Initialization and preparing the data

In [3]:
import nilearn
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

In [4]:
%matplotlib inline 
import seaborn

### load() and get_data() using nibabel

In [5]:
haxby_ds1 = nib.load('/home/farahana/Documents/dataset/Haxby2001/subj1/bold.nii')
haxby_maskvt_ds1 = nib.load('/home/farahana/Documents/dataset/Haxby2001/subj1/mask4_vt.nii')

### using niftimasker to mask the maskvt into sub1 bold.nii

In [6]:
from nilearn.input_data import NiftiMasker
haxby_maskvt_ds1_mask = NiftiMasker(haxby_maskvt_ds1)
haxby_maskedvt_ds1 = haxby_maskvt_ds1_mask.fit_transform(haxby_ds1)

### get the label or target or stimuli from labels.txt using np.recfromcsv

In [7]:
haxby_label1 = np.recfromcsv('/home/farahana/Documents/dataset/Haxby2001/subj1/labels.txt', delimiter=' ')

In [8]:
# separate the chunks and label
haxby_label1_label = haxby_label1['labels']
haxby_label1_chunks = haxby_label1['chunks']

### print the labels, shape of after niftimasker, shape of the labels

In [9]:
print (haxby_label1_label)
print (haxby_maskedvt_ds1.shape)
print (haxby_label1_label.shape)

[b'rest' b'rest' b'rest' ..., b'rest' b'rest' b'rest']
(1452, 577)
(1452,)


## 2. Building the simple Support vector Classification (SVC)

### cross validation and fitting

In [10]:
from sklearn.cross_validation import cross_val_score, KFold
cv = KFold(n=len(haxby_maskedvt_ds1), n_folds=10)

In [11]:
from sklearn.svm import SVC
svc_classifier = SVC(kernel='linear')
svc_classifier.fit(haxby_maskedvt_ds1, haxby_label1_label)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [12]:
cv_score = cross_val_score(svc_classifier, haxby_maskedvt_ds1, haxby_label1_label, cv=cv)
print (cv_score)

[ 0.71232877  0.7260274   0.8137931   0.83448276  0.82758621  0.76551724
  0.71034483  0.64827586  0.56551724  0.66206897]
